In [1]:
import pandas as pd
import torch
from torch import nn
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


c:\Users\User\Desktop\Crochet_Project\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../Data/Train.csv")
df = df.fillna(0)

df.head()


,text,product,height,width,colors,character
0,I want a shinchan plushie,plushie,0.0,0,0.0,shinchan
1,Make a hello kitty doll,doll,0.0,0,0.0,hello kitty
2,Crochet panda plushie around 25 cm,plushie,25.0,0,0.0,0
3,Small bunny plushie 18 cm,plushie,18.0,0,0.0,0
4,I want a teddy bear plushie 30 cm with 3 colors,plushie,30.0,0,3.0,teddy bear


In [3]:
df["product"] = df["product"].astype(str)
df["character"] = df["character"].astype(str)

product_encoder = LabelEncoder()
character_encoder = LabelEncoder()

df["product_enc"] = product_encoder.fit_transform(df["product"])
df["character_enc"] = character_encoder.fit_transform(df["character"])

features = df[["product_enc","height","width","colors","character_enc"]]
texts = df["text"].tolist()


In [4]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

tokens = tokenize(texts)


In [5]:
class CrochetTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(768, 5)   # product, height, width, colors, character

    def forward(self, ids, mask):
        x = self.bert(ids, attention_mask=mask).last_hidden_state[:,0]
        return self.fc(x)

model = CrochetTransformer()


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 260.79it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_projector.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
features = df[["product_enc","height","width","colors","character_enc"]]

features = features.apply(pd.to_numeric, errors="coerce")
features = features.fillna(0)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features[["height","width","colors"]] = scaler.fit_transform(
    features[["height","width","colors"]]
)

X_ids = tokens["input_ids"]
X_mask = tokens["attention_mask"]

y = torch.tensor(features.values, dtype=torch.float32)


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

In [9]:
for epoch in range(500):
    preds = model(X_ids, X_mask)
    loss = loss_fn(preds, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")


Epoch 1 | Loss: 20.4508
Epoch 2 | Loss: 20.1770
Epoch 3 | Loss: 19.9038
Epoch 4 | Loss: 19.6325
Epoch 5 | Loss: 19.3648
Epoch 6 | Loss: 19.1024
Epoch 7 | Loss: 18.8459
Epoch 8 | Loss: 18.5947
Epoch 9 | Loss: 18.3477
Epoch 10 | Loss: 18.1048
Epoch 11 | Loss: 17.8664
Epoch 12 | Loss: 17.6326
Epoch 13 | Loss: 17.4034
Epoch 14 | Loss: 17.1787
Epoch 15 | Loss: 16.9585
Epoch 16 | Loss: 16.7431
Epoch 17 | Loss: 16.5332
Epoch 18 | Loss: 16.3296
Epoch 19 | Loss: 16.1332
Epoch 20 | Loss: 15.9444
Epoch 21 | Loss: 15.7637
Epoch 22 | Loss: 15.5921
Epoch 23 | Loss: 15.4308
Epoch 24 | Loss: 15.2804
Epoch 25 | Loss: 15.1412
Epoch 26 | Loss: 15.0128
Epoch 27 | Loss: 14.8937
Epoch 28 | Loss: 14.7816
Epoch 29 | Loss: 14.6752
Epoch 30 | Loss: 14.5739
Epoch 31 | Loss: 14.4772
Epoch 32 | Loss: 14.3843
Epoch 33 | Loss: 14.2951
Epoch 34 | Loss: 14.2096
Epoch 35 | Loss: 14.1277
Epoch 36 | Loss: 14.0495
Epoch 37 | Loss: 13.9752
Epoch 38 | Loss: 13.9047
Epoch 39 | Loss: 13.8380
Epoch 40 | Loss: 13.7751
Epoch 41 

In [10]:
torch.save(model.state_dict(), "crochet_transformer.pt")
product_encoder.classes_
character_encoder.classes_


array(['0', 'alphabet', 'baby doll', 'beach', 'bear', 'bow', 'butterfly',
       'cartoon boy', 'cat', 'floral', 'flower', 'flowers', 'heart',
       'hello kitty', 'kids theme', 'layered', 'luxury', 'minimal',
       'moon', 'pastel', 'pastel flowers', 'pikachu', 'princess',
       'rainbow', 'rose', 'shinchan', 'star', 'sunflower', 'teddy bear',
       'unicorn', 'wedding'], dtype=object)

In [11]:
import joblib
joblib.dump(product_encoder,"product_encoder.pkl")
joblib.dump(character_encoder,"character_encoder.pkl")

['character_encoder.pkl']

In [12]:
joblib.dump(scaler, "feature_scaler.pkl")

['feature_scaler.pkl']